GENETIC ALGORITHM FOR CONTINUOUS OPTIMIZATION

Aim: To write a python code to implement genetic algorithm for continuous optimization to minimize or maximize an objective function. 

In [1]:
from numpy.random import randint
from numpy.random import rand

In [9]:
# objective function
def objective(x):
	return x[0]**2.0 + x[1]**2.0

In [2]:
# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
	decoded = list()
	largest = 2**n_bits
	for i in range(len(bounds)):
		# extract the substring
		start, end = i * n_bits, (i * n_bits)+n_bits
		substring = bitstring[start:end]
		# convert bitstring to a string of chars
		chars = ''.join([str(s) for s in substring])
		# convert string to integer
		integer = int(chars, 2)
		# scale integer to desired range
		value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
		# store
		decoded.append(value)
	return decoded

In [3]:
# tournament selection
def selection(pop, scores, k=3):
	# first random selection
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]

In [4]:
# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = randint(1, len(p1)-2)
		# perform crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

In [5]:
# mutation operator
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]

In [7]:
# genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
	# initial population of random bitstring
	pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
	# keep track of best solution
	best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
	# enumerate generations
	for gen in range(n_iter):
		# decode population
		decoded = [decode(bounds, n_bits, p) for p in pop]
		# evaluate all candidates in the population
		scores = [objective(d) for d in decoded]
		# check for new best solution
		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				print(">iteration %d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))
		# select parents
		selected = [selection(pop, scores) for _ in range(n_pop)]
		# create the next generation
		children = list()
		for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = selected[i], selected[i+1]
			# crossover and mutation
			for c in crossover(p1, p2, r_cross):
				# mutation
				mutation(c, r_mut)
				# store for next generation
				children.append(c)
		# replace population
		pop = children
	return [best, best_eval]

In [12]:
# define range for input
bounds = [[-5.0, 5.0], [-5.0, 5.0]]
# define the total iterations
n_iter = 100
# bits per variable
n_bits = 16
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
print(f'Starting genetic algorithm\n')
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
decoded = decode(bounds, n_bits, best)
print(f'\nGenetic algorithm completed\n')
print(f'Best solution: {decoded}')
print(f'Fitness score of the best solution: {score:.5f}')

Starting genetic algorithm

>iteration 0, new best f([-2.047271728515625, -1.97540283203125]) = 8.093538
>iteration 0, new best f([0.15594482421875, -1.57745361328125]) = 2.512679
>iteration 0, new best f([-0.55755615234375, -1.076812744140625]) = 1.470395
>iteration 0, new best f([-0.089263916015625, 0.421295166015625]) = 0.185458
>iteration 1, new best f([-0.089263916015625, 0.108795166015625]) = 0.019804
>iteration 4, new best f([-0.0140380859375, 0.016632080078125]) = 0.000474
>iteration 7, new best f([-0.0189208984375, 0.00213623046875]) = 0.000363
>iteration 8, new best f([-0.0128173828125, 0.000457763671875]) = 0.000164
>iteration 9, new best f([0.009307861328125, 0.006103515625]) = 0.000124
>iteration 11, new best f([-0.00396728515625, 0.0018310546875]) = 0.000019
>iteration 12, new best f([-0.00396728515625, 0.000457763671875]) = 0.000016
>iteration 15, new best f([-0.0030517578125, 0.001983642578125]) = 0.000013
>iteration 16, new best f([-0.00030517578125, 0.001373291015625]